In [ ]:
from NextVisionML import MLContext
from NextVisionML import VarianceFilter, DecisionTreeClfr, OneHotClfr

In [ ]:
#Create an context (Loads data)
mlcontext = MLContext()

In [ ]:
#add feature selection methods
mlcontext.hooks.append(VarianceFilter(mlcontext)) 
#Add data prepocessing methods
mlcontext.train_methods.append(DecisionTreeClfr(mlcontext))
mlcontext.train_methods.append(OneHotClfr(mlcontext))
#select a model type

#start the training process
mlcontext.start_train_process()

In [ ]:
import pandas as pd
import dice_ml
from dice_ml.utils import helpers # helper functions
from sklearn.model_selection import train_test_split

In [ ]:
import hyperopt

In [ ]:
from NextVisionML import load_context

sqlContext, train_with_labels, test_with_labels = load_context()


In [ ]:
test_with_labels

In [ ]:
train_dataset = pd.concat([mlcontext.iter_train_X[2], mlcontext.iter_train_y[2]], axis = 0)
cols = []#["Amb_WindSpeed_Std", "Avg_Precipitation", "Grd_Prod_CosPhi_Avg", "Max_Precipitation", "Min_Precipitation"]
train_dataset = train_dataset.drop(columns=cols)
d = dice_ml.Data(
    dataframe=train_dataset,
    continuous_features= list(train_dataset.drop(columns=["Risk Level"]).columns),
    outcome_name="Risk Level")

# Using sklearn backend
m = dice_ml.Model(model=mlcontext.iter_objs[2]["model"], backend="sklearn")
# Using method=random for generating CFs
exp = dice_ml.Dice(d, m, method="random")

In [ ]:
test_data = mlcontext.iter_test_X[2].drop(columns = cols)

In [ ]:
e1 = exp.generate_counterfactuals(test_data.iloc[:5].copy(), total_CFs=2, desired_class=0)

In [ ]:
e1.visualize_as_dataframe(show_only_changes=False)